In [1]:
import pandas as pd
from Classifier.BTree import BTree
from Classifier.Visualize import visualize_tree
from Classifier.BTreeTraversal import BTreeTraversal
from Classifier.adtReSplit import ReSplit, Choose_leaf, CrossSplit
import pickle
import argparse
import os
import time
import matplotlib.pyplot as plt
import warnings
import scanpy as sc
warnings.filterwarnings("ignore")
import numpy as np

In [2]:
def dfs(node, adata, merge_cutoff, prior_gene):
    if node.key == ('leaf',):
        adata_sub = adata[list(set(node.indices)&set(adata.obs_names)),:]
        print(adata_sub.shape)
        # print(len(node.indices),adata_sub.X.shape)
        # sc.pp.scale(adata_sub, max_value=10)
        # sc.tl.pca(adata_sub, n_comps=10)
        # node.stop = None
        node, bic_list, min_bic_node = Choose_leaf(data=adata_sub,prior_gene=prior_gene,merge_cutoff=merge_cutoff,use_parent=True) 
        return node       
    else:
        node.left = dfs(node.left, adata, merge_cutoff, prior_gene)
        node.right = dfs(node.right, adata, merge_cutoff, prior_gene)
        return node

In [3]:
adt_path = [
            '../SeuratV4/subdata/4_10_ADT.csv',
            '../SeuratV4/subdata/4_11_ADT.csv']#'../SeuratV3/subdata/5_4_ADT.csv',
            
rna_path = [
            '../SeuratV4/subdata/4_10_RNA.h5ad',
            '../SeuratV4/subdata/4_11_RNA.h5ad']#'../SeuratV3/subdata/5_4_RNA.h5ad',
            

output_path = '../output/7_1'
merge_cutoff = 0.1
compact_flag = True
current_tree = None


    
starttime = time.time()
print('read data and run CITE-sort.')

adtdata = {}
rnadata = {}
for i in range(len(adt_path)):
    if adt_path[i] != None:
        adtdata[i] = pd.read_csv(adt_path[i], header = 0, index_col=0, sep=',')
    rnadata[i] = sc.read_h5ad(rna_path[i])
    rnadata[i].var_names_make_unique()

# prior_gene = dict({'exhausted':['TOX','TOX2','LAG3','BTLA','CBLB','ITCH','NEDD4'],
# 'TCM/TEM':['CCR7','CD44','IL7R','IL15RA','MBD2'],'Th17':['KLRB1','IL23R','CCR6','IL1R1','STAT3']})
prior_gene = {}
if current_tree != None:
    f = open(current_tree+'tree.pickle','rb')
    tree = pickle.load(f)
    f.close()
    data_sub = adtdata.loc[list(set(tree.indices)&set(adtdata.index)),:]
    tree = dfs(tree, data_sub, merge_cutoff,rnadata)
else:
    # nodelist=[BTree(('leaf',)) for i in range(len(adt_path))]
    crossnode = CrossSplit(adtdata.copy(),merge_cutoff, weight=np.ones(len(adt_path)),rnadata=rnadata.copy())
        
    # tree, bic_list, min_bic_node = Choose_leaf(data=data,merge_cutoff=merge_cutoff,rawdata=data.copy(),datarna=data_rna)

def inner_dfs(node, crossnode, i):
    if node != None and crossnode.left != None:
        # print(crossnode.left.nodelist,crossnode.right.nodelist, node.key)
        node.left = inner_dfs(crossnode.left.nodelist[i], crossnode.left, i) 
        node.right = inner_dfs(crossnode.right.nodelist[i], crossnode.right, i)
    return node

for i in range(len(adtdata)):
    tree = inner_dfs(crossnode.nodelist[i], crossnode, i)
    output = output_path+'_'+str(i)
    if not os.path.exists(output):
        os.mkdir(output)
    visualize_tree(tree, adtdata[i], output, 'tree', compact=compact_flag, rnadata=rnadata[i].copy())

    f = open(output+'/tree.pickle','wb')
    pickle.dump(tree,f)
    f.close()

    # print('generate labels.')

    traversal = BTreeTraversal(tree,save_min_BIC=False)
    leaves_labels = traversal.get_leaf_label()
    leaves_labels.to_csv(output + '/leaf_labels.csv')


endtime = time.time()

print('Time using: ', round(endtime-starttime, 3),'secs')
# plt.plot(list(range(len(bic_list))), bic_list)
# plt.savefig('BIC_as_split.png')



read data and run CITE-sort.
best: ('CD14',) in {('CD14',): 11.635797454282544, ('CLEC12A',): 2.285422750508424, ('CD8',): 2.0107387842493285}
Epoch 1
-------------------------------
loss: 0.010172  [ 9100/ 9129] 2 train data
loss: 0.003586  [ 9100/ 9129] 2 train data
loss: 0.002157  [ 9100/ 9129] 2 train data
Epoch 4
-------------------------------
loss: 0.001549  [ 9100/ 9129] 2 train data
loss: 0.001214  [ 9100/ 9129] 2 train data
loss: 0.001002  [ 9100/ 9129] 2 train data
Epoch 7
-------------------------------
loss: 0.000856  [ 9100/ 9129] 2 train data
loss: 0.000750  [ 9100/ 9129] 2 train data
loss: 0.000669  [ 9100/ 9129] 2 train data
Epoch 10
-------------------------------
loss: 0.000606  [ 9100/ 9129] 2 train data
best: ('CD4-1',) in {('CD4-1',): 2.49998687840485, ('CD8',): -7.697689848441698}
Epoch 1
-------------------------------
loss: 2.502965  [ 7100/ 7114] 2 train data
loss: 0.329492  [ 7100/ 7114] 2 train data
loss: 0.036809  [ 7100/ 7114] 2 train data
Epoch 4
--------

In [ ]:
a = {}
a.values()
# a.append(None)
# a.append(None)

dict_values([])

In [ ]:
rnadata[0].var_names

Index(['FO538757.2', 'AP006222.2', 'RP4-669L17.10', 'RP11-206L10.9',
       'LINC00115', 'FAM41C', 'SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1',
       ...
       'FAM19A5', 'RP3-522J7.6', 'CITF22-1A6.3', 'MOV10L1', 'MIR99AHG',
       'AP000223.42', 'DSCR9', 'AP001626.2', 'AP001046.5', 'AC004556.1'],
      dtype='object', length=17009)